# 🏈 NFL Player Movement Prediction - Complete Pipeline

**End-to-End Machine Learning Workflow**

This notebook demonstrates the complete ML pipeline for predicting NFL player movements during pass plays.

---

## 📋 Table of Contents

1. [Setup & Configuration](#1-setup)
2. [Data Loading](#2-data-loading)
3. [Data Exploration](#3-exploration)
4. [Data Preparation](#4-preparation)
5. [Feature Engineering](#5-features)
6. [Train/Val Split](#6-split)
7. [Model Training](#7-training)
8. [Model Evaluation](#8-evaluation)
9. [Predictions](#9-predictions)
10. [Save Results](#10-save)

---

## 1. Setup & Configuration 🔧

Import libraries and set up configuration

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import pickle
from datetime import datetime
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

# Try to import XGBoost (optional)
try:
    from xgboost import XGBRegressor
    HAS_XGBOOST = True
except ImportError:
    HAS_XGBOOST = False
    print("⚠️  XGBoost not installed. Install with: pip install xgboost")

# Set plotting style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully")
print(f"   XGBoost available: {HAS_XGBOOST}")

In [ ]:
# Configuration
class Config:
    """Pipeline configuration"""
    
    # Paths
    DATA_DIR = Path('../data')
    OUTPUT_DIR = Path('../outputs/end_to_end_pipeline')
    
    # Data settings
    USE_SAMPLE = True  # Set to False for full data
    SAMPLE_SIZE = 100000  # Number of rows to sample
    MAX_FILES = 2  # Number of weekly files to load
    
    # Train/Val split
    VAL_SIZE = 0.2  # 20% validation set
    RANDOM_STATE = 42
    
    # Feature engineering flags
    USE_PHYSICS_FEATURES = True
    USE_SPATIAL_FEATURES = True
    USE_TEMPORAL_FEATURES = True  # ⚠️ Only after split!
    USE_NFL_FEATURES = True
    
    # Models to train
    MODELS = ['ridge', 'random_forest']
    if HAS_XGBOOST:
        MODELS.append('xgboost')

# Create config instance
config = Config()

# Create output directory
config.OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Configuration loaded")
print(f"   Data directory: {config.DATA_DIR}")
print(f"   Output directory: {config.OUTPUT_DIR}")
print(f"   Sample size: {config.SAMPLE_SIZE if config.USE_SAMPLE else 'All data'}")
print(f"   Models: {config.MODELS}")

## 2. Data Loading 📂

Load and merge input/output data files from the training set

In [ ]:
def load_data(data_dir, max_files=None, sample_size=None):
    """
    Load and merge input/output CSV files
    
    Args:
        data_dir: Directory containing train data
        max_files: Maximum number of files to load (None = all)
        sample_size: Sample size for faster testing (None = all)
    
    Returns:
        input_df, output_df: Input and output dataframes
    """
    train_dir = data_dir / 'train'
    
    # Get file lists
    input_files = sorted(train_dir.glob('input_*.csv'))
    output_files = sorted(train_dir.glob('output_*.csv'))
    
    if max_files:
        input_files = input_files[:max_files]
        output_files = output_files[:max_files]
    
    print(f"📂 Loading {len(input_files)} input files and {len(output_files)} output files...")
    
    # Load input files
    input_dfs = []
    for file in input_files:
        df = pd.read_csv(file)
        input_dfs.append(df)
        print(f"   ✓ {file.name}: {len(df):,} rows")
    
    input_df = pd.concat(input_dfs, ignore_index=True)
    
    # Load output files
    output_dfs = []
    for file in output_files:
        df = pd.read_csv(file)
        output_dfs.append(df)
    
    output_df = pd.concat(output_dfs, ignore_index=True)
    
    # Sample if requested
    if sample_size and len(input_df) > sample_size:
        print(f"\n🎲 Sampling {sample_size:,} rows from {len(input_df):,}...")
        input_df = input_df.sample(n=sample_size, random_state=42)
        # Filter output to match sampled input using proper keys
        sampled_keys = input_df[['game_id', 'play_id', 'nfl_id', 'frame_id']]
        output_df = output_df.merge(sampled_keys, on=['game_id', 'play_id', 'nfl_id', 'frame_id'])
        output_df = output_df.reset_index(drop=True)
    
    print(f"\n✅ Data loaded successfully")
    print(f"   Input shape: {input_df.shape}")
    print(f"   Output shape: {output_df.shape}")
    
    return input_df, output_df

In [ ]:
# Load data
input_df, output_df = load_data(
    config.DATA_DIR,
    max_files=config.MAX_FILES,
    sample_size=config.SAMPLE_SIZE if config.USE_SAMPLE else None
)

## 3. Data Exploration 🔍

Quick exploration of the loaded data

In [ ]:
print("📊 Input Data Sample:")
display(input_df.head())

print("\n📊 Output Data Sample:")
display(output_df.head())

print("\n📋 Input Columns:")
print(input_df.columns.tolist())

print("\n❓ Missing Values:")
missing = input_df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    for col, count in missing.head(10).items():
        pct = 100 * count / len(input_df)
        print(f"   {col}: {count:,} ({pct:.1f}%)")
else:
    print("   ✓ No missing values")

In [ ]:
# Quick visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Speed distribution
if 's' in input_df.columns:
    axes[0, 0].hist(input_df['s'].dropna(), bins=50, edgecolor='black', alpha=0.7)
    axes[0, 0].set_title('Speed Distribution', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Speed (yards/sec)')
    axes[0, 0].set_ylabel('Frequency')

# Acceleration distribution
if 'a' in input_df.columns:
    axes[0, 1].hist(input_df['a'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='orange')
    axes[0, 1].set_title('Acceleration Distribution', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Acceleration (yards/sec²)')
    axes[0, 1].set_ylabel('Frequency')

# Field positions
if 'x' in input_df.columns and 'y' in input_df.columns:
    sample = input_df.sample(min(5000, len(input_df)))
    axes[1, 0].scatter(sample['x'], sample['y'], alpha=0.3, s=1, c='blue')
    axes[1, 0].set_title('Player Positions on Field', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('X (yards)')
    axes[1, 0].set_ylabel('Y (yards)')
    axes[1, 0].set_xlim(0, 120)
    axes[1, 0].set_ylim(0, 53.3)

# Target distribution
if 'x' in output_df.columns:
    axes[1, 1].hist(output_df['x'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='green')
    axes[1, 1].set_title('Target X Distribution', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Future X Position (yards)')
    axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'data_exploration.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualizations saved")

## 4. Data Preparation 🧹

Clean, merge, and prepare data for modeling

In [ ]:
def prepare_data(input_df, output_df):
    """
    Prepare data for modeling:
    1. Merge input and output
    2. Handle missing values
    3. Fix data types
    4. Remove outliers
    
    Returns:
        merged_df: Prepared dataframe
    """
    print("="*70)
    print("DATA PREPARATION")
    print("="*70)
    
    # 1. Merge input and output
    print("\n1️⃣ Merging input and output data...")
    merged_df = input_df.merge(
        output_df[['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y']],
        on=['game_id', 'play_id', 'nfl_id', 'frame_id'],
        suffixes=('', '_target')
    )
    merged_df = merged_df.rename(columns={'x_target': 'target_x', 'y_target': 'target_y'})
    print(f"   ✓ Merged shape: {merged_df.shape}")
    
    # 2. Handle missing values
    print("\n2️⃣ Handling missing values...")
    initial_nulls = merged_df.isnull().sum().sum()
    
    # Fill numeric columns with median
    numeric_cols = merged_df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if merged_df[col].isnull().any():
            median_val = merged_df[col].median()
            merged_df[col].fillna(median_val, inplace=True)
    
    # Fill categorical columns with mode
    cat_cols = merged_df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        if merged_df[col].isnull().any():
            mode_val = merged_df[col].mode()[0] if len(merged_df[col].mode()) > 0 else 'Unknown'
            merged_df[col].fillna(mode_val, inplace=True)
    
    final_nulls = merged_df.isnull().sum().sum()
    print(f"   ✓ Nulls: {initial_nulls} → {final_nulls}")
    
    # 3. Remove outliers (using IQR method)
    print("\n3️⃣ Removing outliers...")
    initial_rows = len(merged_df)
    
    for col in ['s', 'a', 'target_x', 'target_y']:
        if col in merged_df.columns:
            Q1 = merged_df[col].quantile(0.01)
            Q3 = merged_df[col].quantile(0.99)
            merged_df = merged_df[(merged_df[col] >= Q1) & (merged_df[col] <= Q3)]
    
    final_rows = len(merged_df)
    removed = initial_rows - final_rows
    print(f"   ✓ Removed {removed:,} outliers ({100*removed/initial_rows:.2f}%)")
    
    print(f"\n✅ Data preparation complete")
    print(f"   Final shape: {merged_df.shape}")
    print(f"   Memory: {merged_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    return merged_df

In [ ]:
# Prepare data
prepared_df = prepare_data(input_df, output_df)

print("\n📋 Prepared data sample:")
display(prepared_df.head())

## 5. Feature Engineering ⚙️

Create physics-based, spatial, and NFL-specific features

**Note:** Temporal features are created AFTER train/test split to avoid data leakage!

In [ ]:
def create_physics_features(df):
    """
    Create physics-based features:
    - Velocity components (v_x, v_y)
    - Acceleration components
    - Momentum
    - Kinetic energy
    """
    print("⚙️  Creating physics features...")
    
    # Velocity components
    if 's' in df.columns and 'dir' in df.columns:
        df['velocity_x'] = df['s'] * np.cos(np.radians(df['dir']))
        df['velocity_y'] = df['s'] * np.sin(np.radians(df['dir']))
    
    # Acceleration components
    if 'a' in df.columns and 'dir' in df.columns:
        df['acceleration_x'] = df['a'] * np.cos(np.radians(df['dir']))
        df['acceleration_y'] = df['a'] * np.sin(np.radians(df['dir']))
    
    # Momentum (mass × velocity)
    if 'player_weight' in df.columns and 's' in df.columns:
        df['momentum'] = df['player_weight'] * df['s']
    
    # Kinetic energy (0.5 × mass × velocity²)
    if 'player_weight' in df.columns and 's' in df.columns:
        df['kinetic_energy'] = 0.5 * df['player_weight'] * (df['s'] ** 2)
    
    print(f"   ✓ Added physics features")
    return df


def create_spatial_features(df):
    """
    Create spatial features:
    - Distance to ball landing
    - Field position features
    - Sideline proximity
    """
    print("⚙️  Creating spatial features...")
    
    # Distance to ball landing
    if all(col in df.columns for col in ['x', 'y', 'ball_land_x', 'ball_land_y']):
        df['dist_to_ball'] = np.sqrt(
            (df['x'] - df['ball_land_x'])**2 + 
            (df['y'] - df['ball_land_y'])**2
        )
        df['dx_to_ball'] = df['ball_land_x'] - df['x']
        df['dy_to_ball'] = df['ball_land_y'] - df['y']
    
    # Field position features
    if 'x' in df.columns:
        df['field_position_norm'] = df['x'] / 120.0
        df['in_red_zone'] = (df['x'] <= 20).astype(int)
    
    # Sideline distance
    if 'y' in df.columns:
        df['dist_to_sideline'] = np.minimum(df['y'], 53.3 - df['y'])
        df['sideline_norm'] = df['dist_to_sideline'] / 26.65
    
    print(f"   ✓ Added spatial features")
    return df


def create_nfl_features(df):
    """
    Create NFL-specific features:
    - Player roles
    - Formation indicators
    - Route detection
    """
    print("⚙️  Creating NFL domain features...")
    
    # Encode player roles
    if 'player_role' in df.columns:
        df['is_targeted_receiver'] = (df['player_role'] == 'Targeted Receiver').astype(int)
        df['is_passer'] = (df['player_role'] == 'Passer').astype(int)
        df['is_defensive'] = (df['player_role'] == 'Defensive Coverage').astype(int)
    
    # Route depth
    if 'x' in df.columns:
        df['route_depth'] = df['x'].abs()
    
    # Lateral movement
    if 'y' in df.columns:
        df['lateral_movement'] = df['y'] - 26.65  # Distance from field center
    
    print(f"   ✓ Added NFL domain features")
    return df

In [ ]:
# Apply feature engineering (before split - no temporal features!)
print("="*70)
print("FEATURE ENGINEERING (Pre-Split)")
print("="*70)

if config.USE_PHYSICS_FEATURES:
    prepared_df = create_physics_features(prepared_df)

if config.USE_SPATIAL_FEATURES:
    prepared_df = create_spatial_features(prepared_df)

if config.USE_NFL_FEATURES:
    prepared_df = create_nfl_features(prepared_df)

print(f"\n✅ Feature engineering complete (pre-split)")
print(f"   Total features: {len(prepared_df.columns)}")

## 6. Train/Validation Split 🔀

Split data into training and validation sets using temporal split

In [ ]:
def train_val_split(df, val_size=0.2, random_state=42):
    """
    Split data into train and validation sets
    Uses temporal split if game_id available, otherwise random
    
    Returns:
        train_df, val_df
    """
    print("="*70)
    print("TRAIN/VALIDATION SPLIT")
    print("="*70)
    
    if 'game_id' in df.columns:
        # Temporal split by game
        print("\n📅 Using temporal split by game...")
        unique_games = df['game_id'].unique()
        np.random.seed(random_state)
        np.random.shuffle(unique_games)
        
        split_idx = int(len(unique_games) * (1 - val_size))
        train_games = unique_games[:split_idx]
        val_games = unique_games[split_idx:]
        
        train_df = df[df['game_id'].isin(train_games)].reset_index(drop=True)
        val_df = df[df['game_id'].isin(val_games)].reset_index(drop=True)
        
        print(f"   Train games: {len(train_games)}")
        print(f"   Val games: {len(val_games)}")
    else:
        # Random split
        print("\n🎲 Using random split...")
        val_size_rows = int(len(df) * val_size)
        val_df = df.sample(n=val_size_rows, random_state=random_state)
        train_df = df[~df.index.isin(val_df.index)].reset_index(drop=True)
        val_df = val_df.reset_index(drop=True)
    
    print(f"\n✅ Split complete")
    print(f"   Train: {len(train_df):,} rows ({100*len(train_df)/len(df):.1f}%)")
    print(f"   Val: {len(val_df):,} rows ({100*len(val_df)/len(df):.1f}%)")
    
    return train_df, val_df

In [ ]:
# Split data
train_df, val_df = train_val_split(prepared_df, val_size=config.VAL_SIZE, random_state=config.RANDOM_STATE)

In [ ]:
# Now add temporal features (AFTER split to avoid leakage)
def create_temporal_features(df):
    """
    Create temporal features (changes over time)
    ⚠️ WARNING: Only use AFTER train/test split to avoid data leakage!
    """
    print("⚠️  Creating temporal features (post-split)...")
    
    # Sort for temporal calculations
    if all(col in df.columns for col in ['game_id', 'play_id', 'nfl_id', 'frame_id']):
        df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id']).reset_index(drop=True)
        
        # Position changes
        for col in ['x', 'y']:
            if col in df.columns:
                df[f'{col}_change'] = df.groupby(['game_id', 'play_id', 'nfl_id'])[col].diff()
        
        # Speed and acceleration changes
        for col in ['s', 'a']:
            if col in df.columns:
                df[f'{col}_change'] = df.groupby(['game_id', 'play_id', 'nfl_id'])[col].diff()
        
        # Fill NaN from diff with 0
        change_cols = [c for c in df.columns if '_change' in c]
        df[change_cols] = df[change_cols].fillna(0)
    
    print(f"   ✓ Added temporal features")
    return df

if config.USE_TEMPORAL_FEATURES:
    print("\n" + "="*70)
    print("TEMPORAL FEATURE ENGINEERING (Post-Split)")
    print("="*70 + "\n")
    train_df = create_temporal_features(train_df)
    val_df = create_temporal_features(val_df)
    print(f"\n✅ Temporal features added")

In [ ]:
# Prepare X and y for modeling
def prepare_model_data(df):
    """
    Prepare features (X) and targets (y) for modeling
    """
    # Identify feature columns (exclude IDs and targets)
    exclude_cols = ['game_id', 'play_id', 'nfl_id', 'frame_id', 
                    'target_x', 'target_y', 'player_name', 'player_position']
    
    feature_cols = [col for col in df.columns if col not in exclude_cols and df[col].dtype in ['int64', 'float64']]
    
    X = df[feature_cols].fillna(0)
    y_x = df['target_x'].fillna(df['target_x'].median())
    y_y = df['target_y'].fillna(df['target_y'].median())
    
    return X, y_x, y_y, feature_cols

# Prepare training data
X_train, y_train_x, y_train_y, feature_names = prepare_model_data(train_df)

# Prepare validation data
X_val, y_val_x, y_val_y, _ = prepare_model_data(val_df)

print(f"✅ Model data prepared")
print(f"   X_train: {X_train.shape}")
print(f"   X_val: {X_val.shape}")
print(f"   Features: {len(feature_names)}")
print(f"\n📋 Sample features: {feature_names[:10]}")

## 7. Model Training 🤖

Train multiple models to predict future player positions

In [ ]:
def train_model(X_train, y_train, X_val, y_val, model_type='ridge'):
    """
    Train a model and evaluate on validation set
    
    Args:
        X_train, y_train: Training data
        X_val, y_val: Validation data
        model_type: Type of model ('ridge', 'random_forest', 'xgboost')
    
    Returns:
        model: Trained model
        metrics: Dictionary of performance metrics
    """
    print(f"\n🤖 Training {model_type.upper()} model...")
    
    # Select model
    if model_type == 'ridge':
        model = Ridge(alpha=1.0, random_state=42)
    elif model_type == 'random_forest':
        model = RandomForestRegressor(
            n_estimators=100,
            max_depth=15,
            min_samples_split=10,
            random_state=42,
            n_jobs=-1
        )
    elif model_type == 'xgboost' and HAS_XGBOOST:
        model = XGBRegressor(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            random_state=42,
            n_jobs=-1
        )
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    # Train
    model.fit(X_train, y_train)
    
    # Predict
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    
    # Calculate metrics
    metrics = {
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'val_rmse': np.sqrt(mean_squared_error(y_val, y_pred_val)),
        'train_mae': mean_absolute_error(y_train, y_pred_train),
        'val_mae': mean_absolute_error(y_val, y_pred_val),
        'train_r2': r2_score(y_train, y_pred_train),
        'val_r2': r2_score(y_val, y_pred_val)
    }
    
    print(f"   ✓ Train RMSE: {metrics['train_rmse']:.4f}")
    print(f"   ✓ Val RMSE: {metrics['val_rmse']:.4f}")
    print(f"   ✓ Val R²: {metrics['val_r2']:.4f}")
    
    return model, metrics, y_pred_val

In [ ]:
# Train models for X coordinate
print("="*70)
print("TRAINING MODELS - X COORDINATE")
print("="*70)

models_x = {}
predictions_x = {}
metrics_x = {}

for model_name in config.MODELS:
    model, metrics, y_pred = train_model(X_train, y_train_x, X_val, y_val_x, model_type=model_name)
    models_x[model_name] = model
    metrics_x[model_name] = metrics
    predictions_x[model_name] = y_pred

print("\n✅ X coordinate models trained")

In [ ]:
# Train models for Y coordinate
print("="*70)
print("TRAINING MODELS - Y COORDINATE")
print("="*70)

models_y = {}
predictions_y = {}
metrics_y = {}

for model_name in config.MODELS:
    model, metrics, y_pred = train_model(X_train, y_train_y, X_val, y_val_y, model_type=model_name)
    models_y[model_name] = model
    metrics_y[model_name] = metrics
    predictions_y[model_name] = y_pred

print("\n✅ Y coordinate models trained")

## 8. Model Evaluation 📊

Compare model performance and visualize results

In [ ]:
# Compare models
print("="*70)
print("MODEL COMPARISON")
print("="*70)

# Create comparison DataFrame
comparison_data = []
for model_name in config.MODELS:
    comparison_data.append({
        'Model': model_name.upper(),
        'X_RMSE': metrics_x[model_name]['val_rmse'],
        'Y_RMSE': metrics_y[model_name]['val_rmse'],
        'X_MAE': metrics_x[model_name]['val_mae'],
        'Y_MAE': metrics_y[model_name]['val_mae'],
        'X_R2': metrics_x[model_name]['val_r2'],
        'Y_R2': metrics_y[model_name]['val_r2']
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n📊 Model Performance Comparison:")
display(comparison_df)

# Find best model
best_x_model = comparison_df.loc[comparison_df['X_RMSE'].idxmin(), 'Model']
best_y_model = comparison_df.loc[comparison_df['Y_RMSE'].idxmin(), 'Model']

print(f"\n🏆 Best model for X: {best_x_model}")
print(f"🏆 Best model for Y: {best_y_model}")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# RMSE comparison
x_rmse = [metrics_x[m]['val_rmse'] for m in config.MODELS]
y_rmse = [metrics_y[m]['val_rmse'] for m in config.MODELS]
model_labels = [m.upper() for m in config.MODELS]

x_pos = np.arange(len(model_labels))
width = 0.35

axes[0].bar(x_pos - width/2, x_rmse, width, label='X Coordinate', alpha=0.8)
axes[0].bar(x_pos + width/2, y_rmse, width, label='Y Coordinate', alpha=0.8)
axes[0].set_xlabel('Model', fontsize=12)
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_title('Model RMSE Comparison', fontsize=14, fontweight='bold')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(model_labels)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# R² comparison
x_r2 = [metrics_x[m]['val_r2'] for m in config.MODELS]
y_r2 = [metrics_y[m]['val_r2'] for m in config.MODELS]

axes[1].bar(x_pos - width/2, x_r2, width, label='X Coordinate', alpha=0.8, color='orange')
axes[1].bar(x_pos + width/2, y_r2, width, label='Y Coordinate', alpha=0.8, color='green')
axes[1].set_xlabel('Model', fontsize=12)
axes[1].set_ylabel('R² Score', fontsize=12)
axes[1].set_title('Model R² Comparison', fontsize=14, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(model_labels)
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Model comparison visualizations saved")

## 9. Predictions & Visualization 🎯

Generate predictions and visualize results

In [ ]:
# Select best models
best_model_x = models_x[best_x_model.lower()]
best_model_y = models_y[best_y_model.lower()]
best_pred_x = predictions_x[best_x_model.lower()]
best_pred_y = predictions_y[best_y_model.lower()]

# Visualize predictions
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# X predictions vs actual
axes[0, 0].scatter(y_val_x, best_pred_x, alpha=0.3, s=1)
axes[0, 0].plot([y_val_x.min(), y_val_x.max()], [y_val_x.min(), y_val_x.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Actual X', fontsize=12)
axes[0, 0].set_ylabel('Predicted X', fontsize=12)
axes[0, 0].set_title(f'X Predictions ({best_x_model})', fontsize=14, fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# Y predictions vs actual
axes[0, 1].scatter(y_val_y, best_pred_y, alpha=0.3, s=1, color='orange')
axes[0, 1].plot([y_val_y.min(), y_val_y.max()], [y_val_y.min(), y_val_y.max()], 'r--', lw=2)
axes[0, 1].set_xlabel('Actual Y', fontsize=12)
axes[0, 1].set_ylabel('Predicted Y', fontsize=12)
axes[0, 1].set_title(f'Y Predictions ({best_y_model})', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# X residuals
residuals_x = y_val_x - best_pred_x
axes[1, 0].hist(residuals_x, bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Residual (Actual - Predicted)', fontsize=12)
axes[1, 0].set_ylabel('Frequency', fontsize=12)
axes[1, 0].set_title('X Residual Distribution', fontsize=14, fontweight='bold')

# Y residuals
residuals_y = y_val_y - best_pred_y
axes[1, 1].hist(residuals_y, bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Residual (Actual - Predicted)', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Y Residual Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'predictions_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Prediction visualizations saved")

## 10. Save Results 💾

Save trained models and results

In [ ]:
# Save models
print("💾 Saving models and results...")

# Save best models
with open(config.OUTPUT_DIR / 'best_model_x.pkl', 'wb') as f:
    pickle.dump(best_model_x, f)

with open(config.OUTPUT_DIR / 'best_model_y.pkl', 'wb') as f:
    pickle.dump(best_model_y, f)

# Save feature names
with open(config.OUTPUT_DIR / 'feature_names.pkl', 'wb') as f:
    pickle.dump(feature_names, f)

# Save metrics
comparison_df.to_csv(config.OUTPUT_DIR / 'model_comparison.csv', index=False)

# Save summary
summary = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'best_model_x': best_x_model,
    'best_model_y': best_y_model,
    'x_rmse': float(comparison_df.loc[comparison_df['Model'] == best_x_model, 'X_RMSE'].values[0]),
    'y_rmse': float(comparison_df.loc[comparison_df['Model'] == best_y_model, 'Y_RMSE'].values[0]),
    'num_features': len(feature_names),
    'train_size': len(train_df),
    'val_size': len(val_df)
}

import json
with open(config.OUTPUT_DIR / 'summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\n✅ Results saved to: {config.OUTPUT_DIR}")
print("\n📋 Summary:")
for key, value in summary.items():
    print(f"   {key}: {value}")

---

## 🎉 Pipeline Complete!

### Summary:
- ✅ Data loaded and prepared
- ✅ Features engineered (physics, spatial, temporal, NFL-specific)
- ✅ Models trained and evaluated
- ✅ Results saved and visualized

### Next Steps:
1. Explore `02_data_exploration.ipynb` for deeper data analysis
2. Try `03_feature_engineering.ipynb` for advanced feature creation
3. Compare more models in `04_model_comparison.ipynb`
4. Experiment with LSTM in `05_lstm_sequence_modeling.ipynb`
5. Generate final predictions in `06_prediction_and_evaluation.ipynb`

---